# Membuat Model Analisis Sentimen



# 1. Import Library yang Dibutuhkan

In [58]:
!pip install gensim
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 31.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.23.5 which is inc

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import joblib
from google.colab import files
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stopwords_id = stopwords.words('indonesian')  # Mendapatkan stopwords bahasa Indonesia

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 2. Memuat Data

In [5]:
# Upload file dari lokal ke Colab
uploaded = files.upload()

# Setelah file diupload, kamu bisa membacanya dengan pandas
# Gantilah nama file sesuai dengan file yang diupload (contoh: 'reviews_data.csv')
sb_df = pd.read_csv(next(iter(uploaded)))  # Mengambil nama file yang pertama yang diupload
print(sb_df.head())

Saving scrapingsuperbank.csv to scrapingsuperbank (2).csv
                               reviewId         userName  \
0  d76cb0fa-a625-4460-82c4-595116a7740f  Pengguna Google   
1  004b6b60-bdd4-4375-b4e3-c7b432e3c484  Pengguna Google   
2  383b51a3-8d61-45fe-ad30-51dcbd72f103  Pengguna Google   
3  238b4cd5-32db-4b7c-8a4e-3a0088bff3fd  Pengguna Google   
4  c165ca55-d9ff-4ccc-aee5-281125108648  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0  Super sekali appnya,simpel dan sangat mudah di...      5             46   
1  Dari pantauan saya sendiri,apps ini memiliki b...      5             39   
2  Bagus m

# 3. Data Preprocessing (Tekstualisasi dan Pelabelan Data)

In [44]:
def label_sentiment(score, content):
    if score > 3:
        label = 'positif'  # Mengganti 'positive' dengan 'positif'
    elif score < 3:
        label = 'negatif'  # Mengganti 'negative' dengan 'negatif'
    else:
        label = 'netral'   # Mengganti 'neutral' dengan 'netral'

    # Memberikan feedback/informasi tambahan
    print(f"Review: {content[:100]}...")  # Menampilkan 100 karakter pertama dari ulasan
    print(f"Rating: {score} => Sentiment: {label}\n")

    return label

# Terapkan fungsi label_sentiment pada DataFrame
sb_df['label'] = sb_df.apply(lambda row: label_sentiment(row['score'], row['content']), axis=1)
sb_df['text'] = sb_df['content']

# Tampilkan beberapa contoh hasil
print(sb_df[['score', 'label', 'content']].head())


Streaming output truncated to the last 5000 lines.
Rating: 5 => Sentiment: positif

Review: Ok...
Rating: 5 => Sentiment: positif

Review: Bagus...
Rating: 5 => Sentiment: positif

Review: Iya mantap...
Rating: 5 => Sentiment: positif

Review: Bagus...
Rating: 5 => Sentiment: positif

Review: Mantab...
Rating: 5 => Sentiment: positif

Review: Top...
Rating: 5 => Sentiment: positif

Review: Mantap...
Rating: 5 => Sentiment: positif

Review: Ok...
Rating: 5 => Sentiment: positif

Review: EXCELENT...
Rating: 5 => Sentiment: positif

Review: Ok...
Rating: 5 => Sentiment: positif

Review: mantap...
Rating: 5 => Sentiment: positif

Review: Mantap...
Rating: 5 => Sentiment: positif

Review: ok...
Rating: 5 => Sentiment: positif

Review: Good...
Rating: 5 => Sentiment: positif

Review: bagus...
Rating: 5 => Sentiment: positif

Review: Top...
Rating: 5 => Sentiment: positif

Review: Mantap...
Rating: 5 => Sentiment: positif

Review: Keren...
Rating: 5 => Sentiment: positif

Review: Mantabs...
R

# 4. Ekstraksi Fitur dengan TF-Idf

In [45]:
# Inisialisasi TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Transformasi teks dan memberi feedback
print("Fitting and transforming the text data...")

# Fit dan transformasi data teks (sb_df['text'])
X = vectorizer.fit_transform(sb_df['text'])

# Memberikan feedback tentang fitur yang diambil
print(f"Total features extracted: {X.shape[1]}")  # Menampilkan jumlah fitur yang diekstraksi (jumlah kolom)
print(f"Some sample feature names: {vectorizer.get_feature_names_out()[:10]}")  # Menampilkan 10 fitur pertama

# Menampilkan bentuk matriks (banyaknya data review dan fitur yang diekstrak)
print(f"Shape of feature matrix: {X.shape}")

# Target variabel
y = sb_df['label']

# Cek apakah data sudah siap
print(f"Label distribution: {y.value_counts()}")

Fitting and transforming the text data...
Total features extracted: 5000
Some sample feature names: ['00' '000' '02' '03' '04' '05' '06' '07' '08' '09']
Shape of feature matrix: (13609, 5000)
Label distribution: label
positif    10980
negatif     2253
netral       376
Name: count, dtype: int64


# 5. Pembagian Data Training dan Testing


In [8]:
# Membagi data menjadi training set dan test set
print("Splitting data into training and test sets...")

# Splitting data (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Memberikan feedback mengenai pembagian data
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of test samples: {X_test.shape[0]}")

# Menampilkan distribusi label di training set dan testing set
print("\nLabel distribution in the training set:")
print(y_train.value_counts())

print("\nLabel distribution in the testing set:")
print(y_test.value_counts())

Splitting data into training and test sets...
Shape of X_train: (10887, 5000)
Shape of X_test: (2722, 5000)
Number of training samples: 10887
Number of test samples: 2722

Label distribution in the training set:
label
positive    8814
negative    1783
neutral      290
Name: count, dtype: int64

Label distribution in the testing set:
label
positive    2166
negative     470
neutral       86
Name: count, dtype: int64


# 6. Menjalankan Model

In [52]:
# Model 1: SVM dengan TF-IDF
# Pembagian data 80/20 (80% untuk training, 20% untuk testing)
X_train, X_test, y_train, y_test = train_test_split(sb_df['text'], sb_df['label'], test_size=0.2, random_state=42)

# Menambahkan n-grams dalam TF-IDF (1-gram dan 2-gram)
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Menambahkan bigrams
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)  # Fit dan transform data training
X_test_tfidf = tfidf_vectorizer.transform(X_test)  # Hanya transform data testing

print("Training SVM Model with TF-IDF...")

# Inisialisasi dan pelatihan model SVM dengan kernel rbf
svm_model = SVC(kernel='rbf', class_weight='balanced')
svm_model.fit(X_train_tfidf, y_train)

# Prediksi pada data test
svm_pred = svm_model.predict(X_test_tfidf)

# Feedback: Akurasi dan metrik untuk model SVM
print("\nSVM Model Evaluation:")
svm_train_acc = accuracy_score(y_train, svm_model.predict(X_train_tfidf))
svm_test_acc = accuracy_score(y_test, svm_pred)
print(f"Training Accuracy: {svm_train_acc * 100:.2f}%")
print(f"Test Accuracy: {svm_test_acc * 100:.2f}%")

# Classification Report untuk model SVM
print("\nClassification Report (SVM):")
# Menambahkan zero_division=1 untuk menghindari UndefinedMetricWarning
print(classification_report(y_test, svm_pred, zero_division=1))

# Confusion Matrix untuk model SVM
print("\nConfusion Matrix (SVM):")
print(confusion_matrix(y_test, svm_pred))

Training SVM Model with TF-IDF...

SVM Model Evaluation:
Training Accuracy: 96.15%
Test Accuracy: 89.02%

Classification Report (SVM):
              precision    recall  f1-score   support

     negatif       0.71      0.87      0.78       470
      netral       0.14      0.10      0.12        86
     positif       0.96      0.93      0.94      2166

    accuracy                           0.89      2722
   macro avg       0.60      0.63      0.62      2722
weighted avg       0.89      0.89      0.89      2722


Confusion Matrix (SVM):
[[ 410   13   47]
 [  50    9   27]
 [ 118   44 2004]]


In [47]:
# Model 2: RF dengan Word2Vec dan pembagian data 80/20
# Tokenisasi menggunakan gensim
sentences = [simple_preprocess(text) for text in sb_df['text']]

# Ekstraksi fitur menggunakan Word2Vec
word2vec_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1)

# Membuat fitur dari Word2Vec
def get_word2vec_features(text, model):
    words = simple_preprocess(text)
    vec = np.zeros(100)  # Karena dimensi vektor yang digunakan 100
    count = 0
    for word in words:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    if count > 0:
        vec /= count
    return vec

# Menyusun fitur Word2Vec untuk seluruh dataset
X_word2vec = np.array([get_word2vec_features(text, word2vec_model) for text in sb_df['text']])

# Pembagian data 80/20
X_train_w2v, X_test_w2v, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Inisialisasi dan pelatihan model Random Forest
rf_model_w2v = RandomForestClassifier(n_estimators=100, class_weight='balanced')
rf_model_w2v.fit(X_train_w2v, y_train)

# Prediksi pada data test
rf_pred_w2v = rf_model_w2v.predict(X_test_w2v)

# Feedback: Akurasi dan metrik untuk model Random Forest dengan Word2Vec
print("\nRandom Forest Model with Word2Vec Evaluation:")
rf_train_acc_w2v = accuracy_score(y_train, rf_model_w2v.predict(X_train_w2v))
rf_test_acc_w2v = accuracy_score(y_test, rf_pred_w2v)
print(f"Training Accuracy: {rf_train_acc_w2v * 100:.2f}%")
print(f"Test Accuracy: {rf_test_acc_w2v * 100:.2f}%")



Random Forest Model with Word2Vec Evaluation:
Training Accuracy: 99.55%
Test Accuracy: 88.46%


In [48]:
# Model 3: RF dengan TF-IDF dan pembagian data 70/30
print("\nTraining Random Forest Model with TF-IDF (70/30 Split)...")

# Pembagian data 70/30
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Inisialisasi dan pelatihan model Random Forest
rf_model_tfidf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
rf_model_tfidf.fit(X_train_tfidf, y_train)

# Prediksi pada data test
rf_pred_tfidf = rf_model_tfidf.predict(X_test_tfidf)

# Feedback: Akurasi dan metrik untuk model Random Forest dengan TF-IDF
print("\nRandom Forest Model with TF-IDF Evaluation:")
rf_train_acc_tfidf = accuracy_score(y_train, rf_model_tfidf.predict(X_train_tfidf))
rf_test_acc_tfidf = accuracy_score(y_test, rf_pred_tfidf)
print(f"Training Accuracy: {rf_train_acc_tfidf * 100:.2f}%")
print(f"Test Accuracy: {rf_test_acc_tfidf * 100:.2f}%")


# Classification Report untuk model Random Forest
print("\nClassification Report (Random Forest):")
print(classification_report(y_test, rf_pred_tfidf, zero_division=1))

# Confusion Matrix untuk model Random Forest
print("\nConfusion Matrix (Random Forest):")
print(confusion_matrix(y_test, rf_pred_tfidf))


Training Random Forest Model with TF-IDF (70/30 Split)...

Random Forest Model with TF-IDF Evaluation:
Training Accuracy: 99.49%
Test Accuracy: 89.69%

Classification Report (Random Forest):
              precision    recall  f1-score   support

     negatif       0.78      0.70      0.74       678
      netral       0.09      0.01      0.02       115
     positif       0.92      0.97      0.94      3290

    accuracy                           0.90      4083
   macro avg       0.60      0.56      0.57      4083
weighted avg       0.87      0.90      0.88      4083


Confusion Matrix (Random Forest):
[[ 475    1  202]
 [  38    1   76]
 [  95    9 3186]]


**Insight**

Dari tiga model yang diuji menggunakan teknik pengolahan teks yang berbeda (TF-IDF, Word2Vec), berikut adalah hasil evaluasi dan kesimpulannya:

1. Model 1: SVM dengan TF-IDF
* Akurasi pelatihan: 96.34%
* Akurasi pengujian: 89.42%

* Model ini memberikan hasil yang cukup baik dengan akurasi yang tinggi. Precision untuk kelas 'positive' sangat baik, tetapi precision untuk kelas 'neutral' masih rendah.

* Confusion Matrix menunjukkan bahwa model ini kesulitan membedakan antara 'neutral' dan kelas lainnya, tetapi sangat baik dalam mengklasifikasikan 'positive'.

2. Model 2: Random Forest dengan Word2Vec

* Akurasi pelatihan: 99.66%
* Akurasi pengujian: 88.39%

* Meskipun model ini memiliki akurasi pelatihan yang sangat tinggi, akurasi pengujian lebih rendah jika dibandingkan dengan SVM. Ini menunjukkan bahwa model ini mungkin mengalami overfitting pada data pelatihan.

* Precision dan recall untuk kelas 'neutral' sangat rendah, sementara model ini sangat efektif dalam mengidentifikasi kelas 'positive'.

3. Model 3: Random Forest dengan TF-IDF (Pembagian 70/30)

* Akurasi pelatihan: 99.41%
* Akurasi pengujian: 89.69%

* Model ini menghasilkan hasil terbaik untuk akurasi pengujian, meskipun precision untuk kelas 'neutral' masih sangat rendah.

* Confusion matrix menunjukkan bahwa model ini cukup efektif dalam mengidentifikasi 'positive' dan 'negative', namun kesulitan dalam mengidentifikasi kelas 'neutral'.

# 8. Fungsi Inferensi

**Menyimpan model SVM dan RF ke dalam file**

In [53]:
# Menyimpan model SVM, Random Forest (dengan Word2Vec), dan Random Forest (dengan TF-IDF) untuk digunakan pada inferensi nanti
joblib.dump(svm_model, 'svm_model.pkl')  # Model SVM dengan TF-IDF
joblib.dump(rf_model_w2v, 'rf_model_w2v.pkl')  # Model Random Forest dengan Word2Vec
joblib.dump(rf_model_tfidf, 'rf_model_tfidf.pkl')  # Model Random Forest dengan TF-IDF

# Menyimpan vectorizer (TF-IDF) untuk digunakan pada inferensi nanti
joblib.dump(tfidf_vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

In [80]:
# Fungsi untuk melakukan prediksi sentimen berdasarkan model dan vectorizer
def predict_sentiment(text, model, vectorizer):
    """
    Fungsi ini menerima input teks, model yang telah dilatih, dan vectorizer (seperti TF-IDF),
    kemudian mengembalikan prediksi sentimen dari model.

    Args:
    - text: string, teks input untuk diprediksi.
    - model: model yang telah dilatih (misalnya SVM atau Random Forest).
    - vectorizer: vectorizer yang digunakan untuk mengubah teks menjadi fitur.

    Returns:
    - string: Prediksi sentimen (positive, negative, neutral)
    """
    # Transformasi teks menggunakan vectorizer
    text_vectorized = vectorizer.transform([text])

    # Melakukan prediksi menggunakan model yang dilatih
    prediction = model.predict(text_vectorized)

    # Mengembalikan hasil prediksi
    return prediction[0]

**Uji Inferensi dengan Teks Sample**

In [81]:
# Sample Texts - Prediksi Sentimen Aplikasi
sample_text = "Saya sangat senang dengan aplikasi ini! Kualitasnya luar biasa dan pelayanan sangat ramah. Pasti akan menggunakannya lagi!"
print(f"Sentimen Prediksi: {predict_sentiment(sample_text, svm_model, vectorizer)}")

sample_text2 = "Aplikasi ini memiliki banyak bug pada fitur, juga susah untuk digunakan."
print(f"Sentimen Prediksi: {predict_sentiment(sample_text2, svm_model, vectorizer)}")

Sentimen Prediksi: positif
Sentimen Prediksi: negatif
